In [1]:
!pip install python-dotenv
!pip install langchain-text-splitters
!pip install langchain langchain-core langchain-community langchain-openai


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\cjdfk\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\cjdfk\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\cjdfk\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [16]:
!pip install pypdf
!pip install beautifulsoup4 
!pip install faiss-cpu


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\cjdfk\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\cjdfk\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.6 MB 1.3 MB/s eta 0:00:12
    --------------------------------------- 0.2/14.6 MB 3.5 MB/s eta 0:00:05
   - -------------------------------------- 0.6/14.6 MB 5.7 MB/s eta 0:00:03
   --- ------------------------------------ 1.3/14.6 MB 8.5 MB/s eta 0:00:02
   ----- ---------------------------------- 2.0/14.6 MB 9.7 MB/s eta 0:00:02
   ------- -------------------------------- 2.7/14.6 MB 10.9 MB/s eta 0:00:02
   --------- ------------------------------ 3.4/14.6 MB 11.6 MB/s eta 0:00:01
   ----------- ---------------------------- 4.1/14.6 MB 11.8 MB/s eta 0:00:01
   ------------ --------------------------- 4.7/14.6 MB 12.1 MB/s eta 0:00:01
   --------------- ------------------------ 5.5/14.6 MB 12.6 MB/s eta 0:00:01
   ----------------- ---------------------- 6.5/14.6 MB 13.3 MB/s eta 0:00:01
   -------------------- ------------------- 7.5/14.6 MB 14.1 MB/s eta 0:00:01


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\cjdfk\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [19]:
import os
from dotenv import load_dotenv
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from operator import itemgetter

In [20]:
# 환경 변수 로드
#load_dotenv()
#OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# api-key.txt 파일에서 API 키를 읽어오기
with open('api-key.txt', 'r') as file:
    OPENAI_API_KEY = file.read().strip()

# 환경 변수로 설정
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [21]:
# 단계 1: 문서 로드(Load Documents)

# PDF 문서 로드
pdf_loader = PyPDFLoader("./data/전자민원.pdf")
pdf_docs = pdf_loader.load()

# URL 문서 로드
url_file_path = "./data/urls.txt"
with open(url_file_path, "r") as file:
    urls = file.read().splitlines()

url_loaders = [WebBaseLoader(url) for url in urls]
url_docs = []
for loader in url_loaders:
    url_docs.extend(loader.load())

# PDF 문서와 URL 문서를 합침
docs = pdf_docs + url_docs
#print(url_docs)

In [22]:
# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)

# 단계 3: 임베딩(Embedding) 생성
embeddings = OpenAIEmbeddings()

# 단계 4: DB 생성(Create DB) 및 저장
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)
vector_store_file = "./data/faiss_index"
vectorstore.save_local(vector_store_file)

# 단계 5: 검색기(Retriever) 생성
retriever = vectorstore.as_retriever()

# 단계 6: 프롬프트 생성(Create Prompt)
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
    Use the following pieces of retrieved context to answer the question. 
    If you don't know the answer, just say that you don't know. 
    Answer in Korean.

#Previous Chat History:
{chat_history}

#Question: 
{question} 

#Context: 
{context} 

#Answer:"""
)

# 단계 7: 언어모델(LLM) 생성
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125", temperature=0)

# 단계 8: 체인(Chain) 생성
chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "chat_history": itemgetter("chat_history"),
    }
    | prompt
    | llm
    | StrOutputParser()
)


# 세션 기록을 저장할 딕셔너리
store = {}

# 세션 ID를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_id):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# 대화를 기록하는 RAG 체인 생성
rag_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,  # 세션 기록을 가져오는 함수
    input_messages_key="question",  # 사용자의 질문이 템플릿 변수에 들어갈 key
    history_messages_key="chat_history",  # 기록 메시지의 키
)

In [24]:
# 첫 번째 질문 실행
first_question = rag_with_history.invoke(
    {"question": "재입국허가 면제 대상은 어떻게 돼?"},
    config={"configurable": {"session_id": "session_1"}},
)

print(first_question)

재입국허가 면제 대상은 등록외국인, 영주 자격자, 재외동포 자격자, 난민여행증명서 소지자 등이 해당합니다.


In [25]:
# 이어진 질문 실행
follow_up_question = rag_with_history.invoke(
    {"question": "면제가 안되면 신청은 어떻게 해야 해?"},
    config={"configurable": {"session_id": "session_1"}},
)
print(follow_up_question)

면제가 안되면 관할 출입국·외국인관서 또는 출장소를 방문하여 신고해야 합니다.


In [26]:
# 이어진 질문 실행
follow_up_question = rag_with_history.invoke(
    {"question": "어떤 서류를 준비하면 돼?"},
    config={"configurable": {"session_id": "session_1"}},
)
print(follow_up_question)

필요한 서류는 신청서, 여권 또는 외국인입국허가서, 거소신고증, 체류지 입증서류입니다.


In [27]:
# 이어진 질문 실행
follow_up_question = rag_with_history.invoke(
    {"question": "더 자세히 말해. 관련된 정보가 쓰여있는 웹페이지가 있어?"},
    config={"configurable": {"session_id": "session_1"}},
)
print(follow_up_question)

해당 정보는 https://www.hikorea.go.kr/cvlappl/CvlapplInfoPageR.pt 에서 확인할 수 있습니다.


In [28]:
# 대화 내역 확인
session_history = get_session_history("session_1")
print("Chat History:")
for message in session_history.messages:
    print(message)

Chat History:
content='유학생이 체류 기간 동안 아르바이트를 하기 위해 어떤 절차를 따라야 해?'
content='유학생이 체류 기간 동안 아르바이트를 하기 위해서는 관할 출입국·외국인관서 또는 출장소를 방문하여 신청해야 합니다.'
content='재입국허가 면제 대상은 어떻게 돼?'
content='재입국허가 면제 대상은 등록외국인, 영주 자격자, 재외동포 자격자, 난민여행증명서 소지자 등이 해당합니다.'
content='면제가 안되면 신청은 어떻게 해야 해?'
content='면제가 안되면 관할 출입국·외국인관서 또는 출장소를 방문하여 신고해야 합니다.'
content='어떤 서류를 준비하면 돼?'
content='필요한 서류는 신청서, 여권 또는 외국인입국허가서, 거소신고증, 체류지 입증서류입니다.'
content='더 자세히 말해. 관련된 정보가 쓰여있는 웹페이지가 있어?'
content='해당 정보는 https://www.hikorea.go.kr/cvlappl/CvlapplInfoPageR.pt 에서 확인할 수 있습니다.'
